In [1]:
DATASET_ID = 'git_repos_call'

In [2]:
import pdb
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import pandas as pd
import librosa
from datetime import datetime
import json
import numpy as np
from pathlib2 import Path

In [3]:
import avgn

In [4]:
from avgn.utils.paths import DATA_DIR, ensure_dir

In [5]:
# create a unique datetime identifier for the files output by this notebook 
##(they will be saved in a folder with today's date/time of output creation)
DT_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
DT_ID

'2022-03-12_17-46-00'

In [6]:
#Defining Path to Raw Dataset
RAW_DATASET_LOC = DATA_DIR/"raw"/"magpie"/"individuals"
RAW_DATASET_LOC

WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals')

In [7]:
#grab an array of all textgrid files 
TGLIST = np.array(list((RAW_DATASET_LOC).expanduser().glob('*/*.TextGrid')))
TGLIST = np.array([i for i in TGLIST if i.stem[0] != '.'])
TGLIST[0], len(TGLIST)

(WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals/BWYa_F_MGGY/BWY MGGY Call Combo 1 290719 PM.TextGrid'),
 153)

In [8]:
#array of all WAV files
WAVLIST = np.array(list((RAW_DATASET_LOC).expanduser().glob('*/*.wav')))
WAVLIST[0], len(WAVLIST)

(WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals/BWYa_F_MGGY/BWY MGGY Call Combo 1 290719 PM.wav'),
 153)

In [9]:
#creating array containing just stem of WAVLIST arrays (i.e. just the file name)
wav_stems = np.array([i.stem for i in WAVLIST])
wav_stems[0]

'BWY MGGY Call Combo 1 290719 PM'

### Import CSV

In [10]:
vocdf = pd.read_csv (DATA_DIR/DATASET_ID/'alldata_call.csv')
vocdf[:3]

,New_ID,indv,gro,location,sex,wavloc,filename,call_label,call_start,call_end,combi_label,combi_start,combi_end,combi_unique_num,call_pos_combi
0,0,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 290719 PM,DSSHDS,0.753604,0.921160,DSSHDS SHSHLH,0.753604,1.367130,0,0
1,1,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 290719 PM,SHSHLH,0.932017,1.367130,DSSHDS SHSHLH,0.753604,1.367130,0,1
2,2,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 300719 AM,DS,1.218085,1.308841,DS USC,1.218085,1.984961,1,0


In [11]:
len(vocdf)

699

In [12]:
#### Need to take WAV location paths from the segment pickle (made in previous step) as they no longer work after 
##being imported from the csv
call_df = pd.read_pickle(DATA_DIR / DATASET_ID / 'walsh_magpie_call_dataframe.pickle')
call_df[:3]

,New_ID,indv,sex,gro,filename,wavloc,tgloc,call_pos_combi,call_start,call_end,call_label
0,0,MGGY,F,BWYa,BWY MGGY Call Combo 1 290719 PM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,0.753604,0.921160,DSSHDS
1,1,MGGY,F,BWYa,BWY MGGY Call Combo 1 290719 PM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,1,0.932017,1.367130,SHSHLH
0,2,MGGY,F,BWYa,BWY MGGY Call Combo 1 300719 AM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,1.218085,1.308841,DS


#### Some checks to make sure the dataframes are the same

In [13]:
len(call_df)

699

In [14]:
len(vocdf)

699

In [15]:
len(vocdf.filename.unique())

153

In [16]:
len(call_df.filename.unique())

153

In [17]:
#Set index to be the same in both dataframes, as per unique identifiers previously created
call_df.set_index('New_ID', inplace=True)

In [18]:
vocdf.set_index('New_ID', inplace=True)

In [19]:
#check 
vocdf.wavloc[20]

'C:\\Users\\slwal\\anaconda3\\envs\\PY36\\avgn_paper-2\\data\\raw\\magpie\\individuals\\BWYa_F_MGGY\\MGGY Alert 2 070817.WAV'

In [20]:
#check
call_df.wavloc[20]

WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals/BWYa_F_MGGY/MGGY Alert 2 070817.WAV')

In [21]:
#check
vocdf.call_label[600]

'SHDS'

In [22]:
#check
call_df.call_label[600]

'SHDS'

In [23]:
#check
vocdf.call_start[601]

163.1926239

In [24]:
#check
call_df.call_start[601]

163.1926239227139

In [25]:
#replace wavloc column in vocdf to be wavloc in call_df
vocdf.loc[:,['wavloc']] = call_df[['wavloc']]

In [26]:
## double check for duplicates in voc df
vocdf[vocdf.index.duplicated()]

,indv,gro,location,sex,wavloc,filename,call_label,call_start,call_end,combi_label,combi_start,combi_end,combi_unique_num,call_pos_combi
New_ID,,,,,,,,,,,,,,


In [27]:
#convert some columns into integers 

vocdf[["call_pos_combi", "combi_unique_num"]] = vocdf[["call_pos_combi", "combi_unique_num"]].apply(pd.to_numeric)

In [28]:
vocdf[:3]

,indv,gro,location,sex,wavloc,filename,call_label,call_start,call_end,combi_label,combi_start,combi_end,combi_unique_num,call_pos_combi
New_ID,,,,,,,,,,,,,,
0,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 290719 PM,DSSHDS,0.753604,0.921160,DSSHDS SHSHLH,0.753604,1.367130,0,0
1,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 290719 PM,SHSHLH,0.932017,1.367130,DSSHDS SHSHLH,0.753604,1.367130,0,1
2,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 300719 AM,DS,1.218085,1.308841,DS USC,1.218085,1.984961,1,0


In [29]:
vocdf.reset_index(inplace=True)

In [30]:
vocdf[3:]

,New_ID,indv,gro,location,sex,wavloc,filename,call_label,call_start,call_end,combi_label,combi_start,combi_end,combi_unique_num,call_pos_combi
3,3,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 300719 AM,USC,1.442686,1.984961,DS USC,1.218085,1.984961,1,1
4,4,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 1 300719 AM,DSSHDS,1.244022,1.420183,DSSHDS LH,1.244022,1.897371,2,0
5,5,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 1 300719 AM,LH,1.471596,1.897371,DSSHDS LH,1.244022,1.897371,2,1
6,6,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 4 210519 PM,DSSHDS,0.306330,0.469851,DSSHDS SHLH,0.306330,0.927264,3,0
7,7,MGGY,BWYa,CRAWLEY,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 4 210519 PM,SHLH,0.483693,0.927264,DSSHDS SHLH,0.306330,0.927264,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,694,MXXGRY,SS,GUILDFORD,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,SS MXXGRY XXXX JUVENILE Call Combination Serie...,LHDS,0.239733,0.438800,SHDS LHDS,0.048004,0.438800,271,1
695,695,MXXGRY,SS,GUILDFORD,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,SS MXXGRY XXXX JUVENILE Call Combination Serie...,SHDS,1.018259,1.148983,SHDS LHDS,1.018259,1.415477,272,0
696,696,MXXGRY,SS,GUILDFORD,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,SS MXXGRY XXXX JUVENILE Call Combination Serie...,LHDS,1.218702,1.415477,SHDS LHDS,1.018259,1.415477,272,1
697,697,MXXGRY,SS,GUILDFORD,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,SS MXXGRY XXXX JUVENILE Call Combination Serie...,SHDS,0.112036,0.238986,SHDS LHDS,0.112036,0.512518,273,0


# Generate JSON from dataframe

In [31]:
from avgn.custom_parsing.magpie_call_for_repos import (
    gen_wav_json
)

In [32]:
savewav=False # don't want to create new WAV files, just reference original ones

In [33]:
with Parallel(n_jobs=-1, verbose=10) as parallel:
    parallel(
        delayed(gen_wav_json)(
            wf,
            vocdf[vocdf.wavloc==wf],
            DT_ID)
        for wf in tqdm(vocdf.wavloc.unique())
    );

  0%|          | 0/153 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1735s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0808s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 146 out of 153 | elapsed:    5.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 153 out of 153 | elapsed:   

### Check it worked

In [34]:
from avgn.utils.hparams import HParams
from avgn.dataset import DataSet

In [35]:
DATASET_ID

'git_repos_call'

In [36]:
DT_ID

'2022-03-12_17-46-00'

In [37]:
# create a set of hyperparameters for processing this dataset.  
hparams = HParams(
    num_mel_bins = 32,
    mel_lower_edge_hertz=400,
    mel_upper_edge_hertz=15000,
    butter_lowcut = 400,
    butter_highcut = 15000,
    ref_level_db = 20,
    min_level_db = -30,
    mask_spec = True,
    win_length_ms = 10,
    hop_length_ms = 2,
    nex=-1,
    n_jobs=-1,
    verbosity = 1,
)

In [38]:
# create a dataset object, which
dataset = DataSet(DATASET_ID, hparams = hparams)

loading json:   0%|          | 0/152 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 152 out of 152 | elapsed:    0.0s finished


getting unique individuals:   0%|          | 0/152 [00:00<?, ?it/s]

In [39]:
# to make sure everything loaded correctly, lets look at a sample JSON
print(json.dumps(dataset.sample_json, indent=4, default=str)[0:15000] + '...')

{
    "filename": "BWY MGGY Call Combo 1 290719 PM",
    "samplerate_hz": 44100,
    "sex": "F",
    "group": "BWYa",
    "location": "CRAWLEY",
    "wav_loc": "C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals/BWYa_F_MGGY/BWY MGGY Call Combo 1 290719 PM.WAV",
    "length_s": 2.1197732426303855,
    "species": "Gymnorhina tibicen dorsalis",
    "common_name": "Western Australian magpie",
    "indvs": {
        "MGGY": {
            "call": {
                "ID": [
                    0.0,
                    1.0
                ],
                "labels": [
                    "DSSHDS",
                    "SHSHLH"
                ],
                "start_times": [
                    0.753604333,
                    0.932017365
                ],
                "end_times": [
                    0.921159875,
                    1.367129904
                ],
                "call_unique_num": [
                    0.0,
                    1.0
            